#### Last update 29.08.2023 (Anna K)

In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib

### Installing dms2dec and RE

In [2]:
!pip install dms2dec

In [3]:
from dms2dec.dms_convert import dms2dec
import re

# S-Bahn Stations

## Scraping

In [4]:
# get the response in the form of html
wikiurl_sbahns="https://de.wikipedia.org/wiki/Liste_der_Stationen_der_S-Bahn_Berlin"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl_sbahns)
print(response.status_code)

200


In [5]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.content, 'html.parser')

indiatable=soup.find('table', {'class':"wikitable"})

In [6]:
df_sbahns=pd.read_html(str(indiatable))
# convert list to dataframe
df_sbahns=pd.DataFrame(df_sbahns[0])
#print(df_sbahns.head())

In [7]:
sstat = []
for station in indiatable.find_all('tr')[1:]:
    stat= station.select("td")[2]
    lines =[]
    for img in stat.select('img'):
        lin = img['alt']
        lines.append(lin)
    sstat.append(lines)
len(sstat)

193

In [8]:
sstat_new = []
for sublist in sstat:
    if len(sublist) == 0:
        sstat_new.append(np.nan)
    else:
        sstat_new.append(sublist)
        
#sstat_new

In [9]:
df_sbahns["Linie(n)"] = sstat_new
#df_sbahns

## Preparing the S-Bahn table

In [10]:
# Create a new df
data_sbahns = df_sbahns[["S-Bahnhof(ehem. Name)", "Linie(n)", "Lage"]]

# Drop stations that were closed (marked with red & have 0 lines)
data_sbahns = data_sbahns[data_sbahns['Linie(n)'].notna()]

# Drop stations that were not yet opened (marked with green (Perleberger Brücke))
data_sbahns = data_sbahns[data_sbahns["S-Bahnhof(ehem. Name)"]!= "Perleberger Brücke"]

# Reset index
data_sbahns.reset_index(inplace = True)

# Rename columns
# Shorten station names
# Split Area - Longitude - Latitude

In [11]:
# Fixing wrong entry

data_sbahns["S-Bahnhof(ehem. Name)"] = data_sbahns["S-Bahnhof(ehem. Name)"].replace({
    "Spandaubis 1997 Spandau West)": "Spandau"})

In [12]:
s_names = []
for rows in data_sbahns["S-Bahnhof(ehem. Name)"]:
    names_row = rows.split("(")[0].strip()
    s_names.append(names_row)
#s_names

In [13]:
data_sbahns["station name"] = s_names
#data_sbahns

In [14]:
def latlon(some_string):
    res = re.sub(r'[A-Za-z()\-öäü]', '', some_string).strip()
    ll = res.split(",")
    if len(res) == 0:
        return None
    lat = dms2dec(ll[0])
    lon = dms2dec(ll[1])
    return lat, lon

In [15]:
#data_sbahns["Lage"].map(lambda x: x[:-27])

In [16]:
areas = data_sbahns["Lage"].map(lambda x: re.sub(r'[^A-Za-z()\-öäü]', '', x)[:-2])
areas = areas.map(lambda x: x.split("(")[0])
#areas

In [17]:
data_sbahns["area"] = areas
#data_sbahns

In [18]:
s_latitude_longitude=data_sbahns["Lage"].map(latlon)
#s_latitude_longitude

In [19]:
#data_sbahns[s_latitude_longitude.isna()]

In [20]:
len(data_sbahns[s_latitude_longitude.isna()])

17

In [21]:
data_sbahns['latitude longitude'] = s_latitude_longitude
data_sbahns["latitude"] = data_sbahns['latitude longitude'].map(lambda x: x[0], na_action='ignore')
data_sbahns["longitude"] = data_sbahns['latitude longitude'].map(lambda x: x[1], na_action='ignore')
#data_sbahns

In [22]:
data_sbahns["lines"] = data_sbahns["Linie(n)"].apply(lambda x: ', '.join(x))
#data_sbahns

In [23]:
sbahns = data_sbahns[['station name', 'lines', 'area', 'latitude', 'longitude']]
#sbahns

# U-Bahn Stations

## Scraping

In [24]:
# get the response in the form of html
wikiurl_ubahns="https://de.wikipedia.org/w/index.php?title=Liste_der_Berliner_U-Bahnh%C3%B6fe&veaction=edit&section=1"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl_ubahns)
print(response.status_code)

200


In [25]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find_all('table',{'class':"wikitable"})


In [26]:
#indiatable[1]

In [27]:
stations = indiatable[1].find_all('tr')[1:]

In [28]:
ustat = []
for station in indiatable[1].find_all('tr')[1:]:
    stat= station.select("td")[1]["data-sort-value"]
    ustat.append(stat)

len(ustat)

200

In [29]:
df_ubahns=pd.read_html(str(indiatable[1]))
# convert list to dataframe
df_ubahns=pd.DataFrame(df_ubahns[0])
#print(df_ubahns.head())

In [30]:
df_ubahns["Linie"] = ustat

In [31]:
data_ubahns = df_ubahns[["Bahnhof (Kürzel) Karte", "Linie", "Ortsteil"]]
#data_ubahns.head(6)

In [32]:
# data_ubahns["Bahnhof (Kürzel) Karte"]

In [33]:
data_ubahns["Bahnhof (Kürzel) Karte"][1]

'Afrikanische Straße (Afr) 52°\xa033′\xa038″\xa0N, 13°\xa020′\xa03″\xa0O'

## Preparing the U-Bahn table

In [34]:
#data_ubahns["Bahnhof (Kürzel) Karte"]

In [35]:
def s_names(some_string):
    name = some_string.split(" (")
    return name[0]
    

In [36]:
station_names = []
for rows in data_ubahns["Bahnhof (Kürzel) Karte"]:
    names_row = rows.split(" (")[0]
    station_names.append(names_row)
#station_names

In [37]:
def latlon(some_string):
    res = re.sub(r'[A-Za-z()]', '', some_string).strip()
    ll = res.split(", ")
    lat = dms2dec(ll[0])
    lon = dms2dec(ll[1])
    return lat, lon

In [38]:
latitude_longitude=data_ubahns["Bahnhof (Kürzel) Karte"].map(latlon)

In [39]:
#latitude_longitude

In [40]:
data_ubahns["station name"] = station_names
data_ubahns["latitude longitude"] = latitude_longitude
data_ubahns["lines"] = data_ubahns["Linie"]
data_ubahns["area"] = data_ubahns["Ortsteil"]
data_ubahns.drop(columns="Bahnhof (Kürzel) Karte", inplace = True)
data_ubahns.drop(columns="Linie", inplace = True)
data_ubahns.drop(columns="Ortsteil", inplace = True)
#data_ubahns

/var/folders/13/cyjcpcpx6zx62lkt8r16y6dh0000gn/T/ipykernel_1726/1692965557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ubahns["station name"] = station_names
/var/folders/13/cyjcpcpx6zx62lkt8r16y6dh0000gn/T/ipykernel_1726/1692965557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ubahns["latitude longitude"] = latitude_longitude
/var/folders/13/cyjcpcpx6zx62lkt8r16y6dh0000gn/T/ipykernel_1726/1692965557.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

In [41]:
ubahn = data_ubahns.groupby("station name").agg({
    "lines": lambda x: ', '.join(x), 
    'latitude longitude': 'first', 
    'area': 'first'}).reset_index()
#ubahn

In [42]:
ubahn['latitude'] = ubahn['latitude longitude'].map(lambda x: x[0])

In [43]:
ubahn['longitude'] = ubahn['latitude longitude'].map(lambda x: x[1])

In [44]:
ubahn.drop(columns="latitude longitude", inplace=True)
#ubahn

In [45]:
ubahn.columns

Index(['station name', 'lines', 'area', 'latitude', 'longitude'], dtype='object')

# Both S- and U-Bahn

In [46]:
sbahns = data_sbahns[['station name', 'lines', 'area', 'latitude', 'longitude']]
sbahns

,station name,lines,area,latitude,longitude
0,Adlershof,"S45, S46, S8, S85, S9",Adlershof,52.434722,13.541389
1,Ahrensfelde,S7,Marzahn,52.571667,13.565000
2,Alexanderplatz,"S3, S5, S7, S9",Mitte,52.521389,13.411944
3,Alt-Reinickendorf,S25,Reinickendorf,52.577778,13.350556
4,Altglienicke,"S45, S9",Altglienicke,52.407222,13.558889
...,...,...,...,...,...
163,Yorckstraße,S1,Schöneberg,52.492222,13.367778
164,Zehlendorf,S1,Zehlendorf,52.430833,13.258333
165,Zepernick,S2,Panketal,52.659722,13.533889
166,Zeuthen,S46,Zeuthen,52.348611,13.627500


In [47]:
ubahns = ubahn[['station name', 'lines', 'area', 'latitude', 'longitude']]
ubahns

,station name,lines,area,latitude,longitude
0,Adenauerplatz,U7,Charlottenburg,52.499722,13.307222
1,Afrikanische Straße,U6,Wedding,52.560556,13.334167
2,Alexanderplatz,"U2, U5, U8",Mitte,52.521389,13.413333
3,Alt-Mariendorf,U6,Mariendorf,52.439722,13.387500
4,Alt-Tegel,U6,Tegel,52.589444,13.283611
...,...,...,...,...,...
173,Wutzkyallee,U7,Gropiusstadt,52.423333,13.474722
174,Yorckstraße,U7,Schöneberg,52.493056,13.370833
175,Zitadelle,U7,Haselhorst,52.537778,13.217778
176,Zoologischer Garten,"U2, U9",Charlottenburg,52.507222,13.332500


In [48]:
#data_sbahns[s_latitude_longitude.isna()]

In [49]:
stations_df = pd.concat([ubahns, sbahns], axis=0)
stations_df[stations_df["station name"].duplicated(keep=False)==True].sort_values(by="station name")

,station name,lines,area,latitude,longitude
2,Alexanderplatz,"U2, U5, U8",Mitte,52.521389,13.413333
2,Alexanderplatz,"S3, S5, S7, S9",Mitte,52.521389,13.411944
23,Brandenburger Tor,"S1, S2, S25, S26",Mitte,52.516389,13.380833
19,Brandenburger Tor,U5,Mitte,52.516389,13.380833
26,Bundesplatz,"S41, S42, S46",Wilmersdorf,52.477500,13.328611
22,Bundesplatz,U9,Wilmersdorf,52.478889,13.328056
34,Frankfurter Allee,"S41, S42, S8, S85",Friedrichshain,52.515000,13.474167
33,Frankfurter Allee,U5,Friedrichshain,52.515000,13.474722
39,Friedrichstraße,"S1, S2, S25, S26, S3, S5, S7, S9",Mitte,52.520278,13.386944
40,Friedrichstraße,U6,Mitte,52.520278,13.386944


In [50]:
stations_df = stations_df.groupby("station name").agg({"lines": lambda x: ', '.join(x), 
                                        'area': "first", 
                                         'latitude': "mean", 
                                         'longitude': "mean"
                                        })

In [51]:
#stations_df

In [52]:
stations_df[stations_df["latitude"].isna()]

,lines,area,latitude,longitude
station name,,,,
Julius-Leber-Brücke,S1,Schöneberg,NaN,NaN
Lankwitz,S25,Lankwitz,NaN,NaN
Lichtenrade,S2,Lichtenrade,NaN,NaN
Lichterfelde Ost,"S25, S26",Lichterfelde,NaN,NaN
Lichterfelde Süd,"S25, S26",Lichterfelde,NaN,NaN
Mahlow,S2,Blankenfelde-Mahl,NaN,NaN
Marienfelde,S2,Marienfelde,NaN,NaN
Oranienburger Straße,"S1, S2, S25, S26",Mitte,NaN,NaN
Osdorfer Straße,S25,Lichterfelde,NaN,NaN


## Filling in the missing Longitudes and Latitudes (manually)

In [53]:
stations_df.loc["Julius-Leber-Brücke", 'latitude'] = 52.486268888889
stations_df.loc["Lankwitz", 'latitude'] = 52.438611111111
stations_df.loc["Lichtenrade", 'latitude'] = 52.387146
stations_df.loc["Lichterfelde Ost", 'latitude'] = 52.43 
stations_df.loc["Lichterfelde Süd", 'latitude'] = 52.41
stations_df.loc["Mahlow", 'latitude'] = 52.360230555556
stations_df.loc["Marienfelde", 'latitude'] = 52.424166666667
stations_df.loc["Oranienburger Straße", 'latitude'] = 52.525046
stations_df.loc["Osdorfer Straße", 'latitude'] = 52.41925
stations_df.loc["Priesterweg", 'latitude'] = 52.4601 
stations_df.loc["Savignyplatz", 'latitude'] = 52.505277777778
stations_df.loc["Schichauweg", 'latitude'] = 52.398547
stations_df.loc["Südende", 'latitude'] = 52.448117
stations_df.loc["Teltow Stadt", 'latitude'] = 52.396878
stations_df.loc["Tiergarten", 'latitude'] = 52.51444

stations_df.loc["Julius-Leber-Brücke", 'longitude'] = 13.360211944444
stations_df.loc["Lankwitz", 'longitude'] = 13.341666666667 
stations_df.loc["Lichtenrade", 'longitude'] = 13.396526
stations_df.loc["Lichterfelde Ost", 'longitude'] = 13.328888888889
stations_df.loc["Lichterfelde Süd", 'longitude'] = 13.308611111111
stations_df.loc["Mahlow", 'longitude'] = 13.408480555556
stations_df.loc["Marienfelde", 'longitude'] = 13.374722222222
stations_df.loc["Oranienburger Straße", 'longitude'] = 13.392924
stations_df.loc["Osdorfer Straße", 'longitude'] = 13.314555555556 
stations_df.loc["Priesterweg", 'longitude'] = 13.3563 
stations_df.loc["Savignyplatz", 'longitude'] = 13.318888888889
stations_df.loc["Schichauweg", 'longitude'] = 13.389372
stations_df.loc["Südende", 'longitude'] = 13.353646
stations_df.loc["Teltow Stadt", 'longitude'] = 13.276392
stations_df.loc["Tiergarten", 'longitude'] = 13.33649



In [54]:
stations_df = stations_df.reset_index()
stations_df

,station name,lines,area,latitude,longitude
0,Adenauerplatz,U7,Charlottenburg,52.499722,13.307222
1,Adlershof,"S45, S46, S8, S85, S9",Adlershof,52.434722,13.541389
2,Afrikanische Straße,U6,Wedding,52.560556,13.334167
3,Ahrensfelde,S7,Marzahn,52.571667,13.565000
4,Alexanderplatz,"U2, U5, U8, S3, S5, S7, S9",Mitte,52.521389,13.412639
...,...,...,...,...,...
313,Zepernick,S2,Panketal,52.659722,13.533889
314,Zeuthen,S46,Zeuthen,52.348611,13.627500
315,Zitadelle,U7,Haselhorst,52.537778,13.217778
316,Zoologischer Garten,"U2, U9, S3, S5, S7, S9",Charlottenburg,52.507222,13.332500


In [55]:
stations_df["station name"] = stations_df["station name"].replace({
    "Rotes Rathaus 52° 31′ 7″ N, 13° 24′ 30″ O": "Rotes Rathaus", 
    "Museumsinsel 52° 31′ 3″ N, 13° 23′ 54″ O": "Museumsinsel"})

In [56]:
stations_df.loc[stations_df["station name"]=="Rotes Rathaus"]

,station name,lines,area,latitude,longitude
231,Rotes Rathaus,U5,Mitte,52.518611,13.408333


In [57]:
#stations_df.loc[stations_df["station name"]=="R Rathaus"]

In [58]:
stations_df.loc[stations_df["station name"]=="Spandau"]

,station name,lines,area,latitude,longitude
258,Spandau,"S3, S9",Spandau,52.534722,13.195833


## Adding the keys (adjusted to the clean telegram dataset)

In [59]:
stations = stations_df["station name"]

In [70]:
stations = [x.replace('ß', 'ss') for x in stations]
stations = [x.replace('-', ' ') for x in stations]
stations = [x.lower() for x in stations]
stations = [x.replace('strasse', 'str') for x in stations]
stations = [x.replace('alexanderplatz', 'alex') for x in stations]
stations = [x.replace('zoologischer garten', 'zoo') for x in stations]

In [71]:
stations_df["keys"] = stations

In [72]:
stations_df

,station name,lines,area,latitude,longitude,keys
0,Adenauerplatz,U7,Charlottenburg,52.499722,13.307222,adenauerplatz
1,Adlershof,"S45, S46, S8, S85, S9",Adlershof,52.434722,13.541389,adlershof
2,Afrikanische Straße,U6,Wedding,52.560556,13.334167,afrikanische str
3,Ahrensfelde,S7,Marzahn,52.571667,13.565000,ahrensfelde
4,Alexanderplatz,"U2, U5, U8, S3, S5, S7, S9",Mitte,52.521389,13.412639,alex
...,...,...,...,...,...,...
313,Zepernick,S2,Panketal,52.659722,13.533889,zepernick
314,Zeuthen,S46,Zeuthen,52.348611,13.627500,zeuthen
315,Zitadelle,U7,Haselhorst,52.537778,13.217778,zitadelle
316,Zoologischer Garten,"U2, U9, S3, S5, S7, S9",Charlottenburg,52.507222,13.332500,zoo


## Saving the final CSV Table

In [73]:
stations_df.to_csv("./s_u_stations_fixed_with_keys.csv")